<h1> Capstone Project</h1>
<h2>- Locating the best alternative for home buyer in Melbourne </h2>


<h3>Problem</h3>
My client will move to Melbourne soon. He likes Doncaster and its neighbourhood. However, he does not have a lot of budget and he would like to see if there are suburbs like Doncaster with lower house price. He is flexible with property type.   

<h3>Data</h3>
<ul>
<li>1. Locate Melbourne Suburbs with longitude and latitude information: https://www.matthewproctor.com/australian_postcodes</li>
        <li>2. Locate Melbourne Suburbs facilities Source 1: https://developer.foursquare.com/ Source 2 (next step): https://www.zomato.com/ </li>
<li>3. Property Price https://developer.domain.com.au/ </li>
</ul>

<h3>Index</h3>
Understanding data<br>
Defining function<br>
Getting Suburb data<br>
Building clusters<br>
Visualizing data<br>
Conclusion

<h4>Defining functions</h4>

In [72]:
def checkfoursquareborough(CLIENT_ID, CLIENT_SECRET, geopd, i, radius):
    ACCESS_TOKEN = '' # your FourSquare Access Token
    VERSION = '20210201' # Foursquare API version
    LIMIT = 100 # A default Foursquare API limit value
    print('Your credentails:')
    print('CLIENT_ID: ' + CLIENT_ID)
    print('CLIENT_SECRET:' + CLIENT_SECRET)
    print('ACCESS_TOKEN:' + ACCESS_TOKEN)

    t=geopd
    #print(t.head(i))
    t.loc[i, 'locality']
    neighborhood_latitude = t.loc[i, 'lat'] # neighborhood latitude value
    neighborhood_longitude = t.loc[i, 'long'] # neighborhood longitude value
    neighborhood_name = t.loc[i, 'locality'] # neighborhood name
    print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                                   neighborhood_latitude, 
                                                                  neighborhood_longitude))
    latitude = neighborhood_latitude 
    longitude = neighborhood_longitude 
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
    url
    return requests.get(url).json()


In [274]:
# modified to include postal code and borough
def getNearbyVenues(CLIENT_ID, CLIENT_SECRET, locality, postal, borough, latitudes, longitudes, radius=500):
    ACCESS_TOKEN = '' # your FourSquare Access Token
    VERSION = '20210201' # Foursquare API version
    LIMIT = 100 # A default Foursquare API limit value
    venues_list=[]
    for name, post, bor, lat, lng in zip(locality, postal, borough, latitudes, longitudes):
        #print(name, post, bor)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        
        try:
        # make the GET request
            results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
            venues_list.append([(
            name, 
            post,
            bor,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        except:
            print("Error in:",name, post, bor)
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Postal Code', 
                  'Borough', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [112]:
# Function to sort venues in decending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

def getTopVenue (nearbyvenues, numvenues=10):
    onehot = pd.get_dummies(nearbyvenues[['Venue Category']], prefix="", prefix_sep="")
    onehot.head(10)

    # add neighborhood column back to dataframe
    onehot['Neighborhood'] = nearbyvenues['Neighborhood'] 
    # move neighborhood column to the first column
    fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
    onehot = onehot[fixed_columns]

    venuegrouped = onehot.groupby(onehot['Neighborhood']).mean().reset_index()

    for hood in venuegrouped['Neighborhood']:
        #print("----"+hood+"----")
        temp = venuegrouped[venuegrouped['Neighborhood'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        #print('\n')
    
    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighborhood']
    for ind in np.arange(numvenues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    neighborhoodsvenuessorted = pd.DataFrame(columns=columns)
    neighborhoodsvenuessorted['Neighborhood'] = venuegrouped['Neighborhood']

    for ind in np.arange(venuegrouped.shape[0]):
        neighborhoodsvenuessorted.iloc[ind, 1:] = return_most_common_venues(venuegrouped.iloc[ind, :], numvenues)

    return neighborhoodsvenuessorted, venuegrouped

In [476]:
#Domain.com.au
import requests
import json

#https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/Pyrmont/2009

def getSuburbPerformance(client_id, client_secret, state, suburb, postcode, category, rooms):
    scopes = ['api_suburbperformance_read']
    auth_url = 'https://auth.domain.com.au/v1/connect/token'
    url_endpoint2 = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/'
    suburbinfolist=[]
    response = requests.post(auth_url, data = {
                        'client_id':client_id,
                        'client_secret':client_secret,
                        'grant_type':'client_credentials',
                        'scope':scopes,
                        'Content-Type':'text/json'
                        })
    json_res = response.json()
    access_token=json_res['access_token']
    #print(access_token)
    auth = {'Authorization':'Bearer ' + access_token}
    #url = url_endpoint2 +state+'/'+suburb+'?periodSize=Years&startingPeriodRelativeToCurrent=1&totalPeriods=1'
    url = url_endpoint2 +state+'/'+suburb+'/'+postcode+'?propertyCategory='+category+'&bedrooms='+str(rooms)+'&periodSize=Years&startingPeriodRelativeToCurrent=1&totalPeriods=1'
    r = requests.get(url, headers=auth).json()   
    try:
       # make the GET request
        results = r['series']['seriesInfo']
       # for v in results:
       #     print(v['values'])

        suburbinfolist.append([(suburb,
                                v['values']['medianSoldPrice'], 
                                v['values']['numberSold'],
                                v['values']['highestSoldPrice'],
                                v['values']['lowestSoldPrice'],
                                v['values']['5thPercentileSoldPrice'],
                                v['values']['25thPercentileSoldPrice'],
                                v['values']['75thPercentileSoldPrice'],
                                v['values']['95thPercentileSoldPrice']
                               ) for v in results])
        #print(suburbinfolist)
    except:
        print("Error in:"+suburb)

    suburbinfo = pd.DataFrame([item for suburbinfolist in suburbinfolist for item in suburbinfolist])
    suburbinfo.columns = ['Neighborhood',
                          'MedianSoldPrice', 
                          'NumberSold',
                          'HighestSoldPrice',
                          'LowestSoldPrice',
                          '5thPercentileSoldPrice',
                          '25thPercentileSoldPrice',
                          '75thPercentileSoldPrice',
                          '95thPercentileSoldPrice' 
                        ]
    #print(suburbinfo)
    return suburbinfo

# modified to include postal code and borough
def getSuburbPerformanceA(client_id, client_secret, state, suburb, postcode, long, lat, category, rooms):
    scopes = ['api_suburbperformance_read']
    auth_url = 'https://auth.domain.com.au/v1/connect/token'
    url_endpoint2 = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/'
    suburbinfolist=[] 
    
    for lsuburb, lpostcode, llong, llat in zip(suburb, postcode, long, lat):
        #print(name, post, bor)
            
        # create the API request URL
        response = requests.post(auth_url, data = {
                        'client_id':client_id,
                        'client_secret':client_secret,
                        'grant_type':'client_credentials',
                        'scope':scopes,
                        'Content-Type':'text/json'
                        })
        json_res = response.json()
        access_token=json_res['access_token']
        #print(access_token)
        auth = {'Authorization':'Bearer ' + access_token}
        #url = url_endpoint2 +state+'/'+lsuburb+'?periodSize=Years&startingPeriodRelativeToCurrent=1&totalPeriods=1'
        url = url_endpoint2 +state+'/'+lsuburb+'/'+lpostcode+'?propertyCategory='+category+'&bedrooms='+str(rooms)+'&periodSize=Years&startingPeriodRelativeToCurrent=1&totalPeriods=1'
        print(url)
        r = requests.get(url, headers=auth).json()  
        
        
        try:
           # make the GET request
            results = r['series']['seriesInfo']
            print(results)
           # for v in results:
           #     print(v['values'])

            suburbinfolist.append([(lsuburb,
                                    llong,
                                    llat,
                                    v['values']['medianSoldPrice'], 
                                    v['values']['numberSold'],
                                    v['values']['highestSoldPrice'],
                                    v['values']['lowestSoldPrice'],
                                    v['values']['5thPercentileSoldPrice'],
                                    v['values']['25thPercentileSoldPrice'],
                                    v['values']['75thPercentileSoldPrice'],
                                    v['values']['95thPercentileSoldPrice']
                                   ) for v in results])
            #print(suburbinfolist)
        except:
            print("Error in:"+suburb)
    
        suburbinfo = pd.DataFrame([item for suburbinfolist in suburbinfolist for item in suburbinfolist])
        suburbinfo.columns = ['Neighborhood',
                              'Longtitude',
                              'Latitude',
                          'MedianSoldPrice', 
                          'NumberSold',
                          'HighestSoldPrice',
                          'LowestSoldPrice',
                          '5thPercentileSoldPrice',
                          '25thPercentileSoldPrice',
                          '75thPercentileSoldPrice',
                          '95thPercentileSoldPrice' 
                        ]
    
    return(suburbinfo)

In [368]:
# distance between 2 points
from math import sin, cos, sqrt, atan2

def caldist(lat1, lon1, lat2, lon2):
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

<h4> Understanding data </h4>

In [5]:
import pandas as pd
import numpy as np
!pip install lxml
!pip install html5lib
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

In [276]:
# 1. Locate Melbourne Suburbs with longitude and latitude information: https://www.matthewproctor.com/australian_postcodes
# The file is downloaded 
# The csv includes Post Code, State, Borough, Locality (neighborhood), Longtitude, Latitude
# As we focus on Inner Mel only, we filter the dataframe further using Borough field
geocodefile = 'australian_postcodes.csv'
geo=pd.read_csv(geocodefile,header=0)
geo.sort_values(by='state', ascending=True)
geo.rename(columns={"sa4name": "borough"}, inplace=True)
#geo.head(5)
#Extract only key info including postcode, state, borough, suburb, longtitude, latitude
ausgeo = geo[["postcode", "state", "borough", "locality", "long", "lat"]]
ausgeoNNA = ausgeo.dropna()
#ausgeoNNA.head()
melgeo = ausgeoNNA[ausgeoNNA['borough'].str.contains('Melbourne - Inner')]
melgeo.head()


,postcode,state,borough,locality,long,lat
6103,3000,VIC,Melbourne - Inner,MELBOURNE,144.956776,-37.817403
6104,3001,VIC,Melbourne - Inner,MELBOURNE,144.956776,-37.817403
6105,3002,VIC,Melbourne - Inner,EAST MELBOURNE,144.982207,-37.818517
6106,3003,VIC,Melbourne - Inner,WEST MELBOURNE,144.949592,-37.810871
6107,3004,VIC,Melbourne - Inner,MELBOURNE,144.970161,-37.844246


In [275]:
#2. Locate Melbourne Suburbs facilities Source 1: https://developer.foursquare.com/
CLIENT_ID = 'JTSGESUOB5CZZF2FZFEK5FA4VR2JESYE1HMA3H5AOTL1PLBX' # your Foursquare ID
CLIENT_SECRET = 'F2X0XHCZHMP2F5NDAMK35AVCHTOY00VJUTGFLXS2TH2M2ZPS' # your Foursquare Secret

results = checkfoursquareborough(CLIENT_ID, CLIENT_SECRET, geopd=ausgeo, i=10, radius=1000)
results

melgeo = ausgeoNNA[ausgeoNNA['borough'].str.contains('Melbourne - Inner')]
tt=melgeo

melvenues = getNearbyVenues(CLIENT_ID, CLIENT_SECRET, locality=tt['locality'],postal=tt['postcode'],borough=tt['borough'],latitudes=tt['lat'],longitudes=tt['long'], radius=500)



Your credentails:
CLIENT_ID: JTSGESUOB5CZZF2FZFEK5FA4VR2JESYE1HMA3H5AOTL1PLBX
CLIENT_SECRET:F2X0XHCZHMP2F5NDAMK35AVCHTOY00VJUTGFLXS2TH2M2ZPS
ACCESS_TOKEN:
Latitude and longitude values of LEE POINT are -12.381806, 130.866242.


In [277]:
melvenuesorted, melvenuegrouped = getTopVenue(melvenues, 10)
#melvenuesorted.head()
melvenuegrouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Casino,Cemetery,Chaat Place,Chinese Restaurant,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Theater,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Event Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Opera House,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Soba Restaurant,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Yunnan Restaurant,Zoo Exhibit
0,ABBOTSFORD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.190476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.047619,0.047619,0.0,0.0,0.0,0.0,0.0

In [272]:
# Understand domain.com.au data
# we use the getsuburbperformance API to obtain selling price for the specific suburb, category and #room
# we message the data to include highest, lowest, 5,25,75,95 percentile for reference
# doc in https://developer.domain.com.au/docs/v2/apis/pkg_properties_locations/references/suburbperformance_get_bynamedsuburb
client_id = 'client_b61f3b00ce3bd753a8c9931930366271'
client_secret = 'secret_a53535017d9cbe2adf095eb107288489'
doncasterinfo = getSuburbPerformance(client_id, client_secret, state='VIC', suburb='DONCASTER', postcode='3108', category='House', rooms=3)
doncasterinfo.head()

,Neighborhood,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,5thPercentileSoldPrice,25thPercentileSoldPrice,75thPercentileSoldPrice,95thPercentileSoldPrice
0,DONCASTER,1050000,49,1745000,595000,795000,945000,1220000,1375000


<h4>Getting Suburb data</h4>

In [341]:
#Extract only Inner Melbourne borough
melgeo = ausgeoNNA[ausgeoNNA['borough'].str.match('Melbourne - Inner')]
melgeo=melgeo.sort_values(by='locality', ascending=True)
melgeo.groupby(['borough']).count()
#melgeo.head()

,postcode,state,locality,long,lat
borough,,,,,
Melbourne - Inner,85,85,85,85,85
Melbourne - Inner East,41,41,41,41,41
Melbourne - Inner South,72,72,72,72,72


<h4>Building clusters</h4>

In [467]:
#target neighborhood
targetneighborhood = 'DONCASTER'
#k-mean
# set number of clusters
kclusters = 20

melgroupedclustering = melvenuegrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(melgroupedclustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:kclusters] 

melvenuesorted.head()

j=melgeo
j=j.rename(columns={'locality':'Neighborhood', 'postcode':'PostCode', 'long':'Long', 'lat':'Lat'})
j.head()

j_grouped=j.groupby(j['Neighborhood']).mean().reset_index()
print(j_grouped.shape)
j_grouped.head()
#print(neighborhoods_venues_sorted.shape)
melvenuesorted.head()
# add clustering labels
melvenuesorted.drop(['Cluster Labels'], axis=1, errors='ignore', inplace=True)
melvenuesorted.insert(0, 'Cluster Labels', kmeans.labels_)
melvenuesorted.head()
# join
melmerged = j_grouped
melmerged = melmerged.join(melvenuesorted.set_index('Neighborhood'), on='Neighborhood')

#melmerged.head() # check the last columns!
#melmerged.columns
melmerged=melmerged.dropna()
melmerged.reset_index()
#melmerged.head()
melmerged['Cluster Labels']=melmerged['Cluster Labels'].astype('int64') 
#melmerged.head(100)

#our customer desire to move to Doncaster, we will pull out this information first
targetclusterpd = melmerged.loc[melmerged['Neighborhood'] == targetneighborhood]
targetcluster = targetclusterpd.iloc[0]['Cluster Labels']
targetlong  = targetclusterpd.iloc[0]['Long']
targetlat  = targetclusterpd.iloc[0]['Lat']
#find all similar neighborhood with cluster #5
neighborhoodalike=melmerged.loc[melmerged['Cluster Labels'] == targetcluster]
#print(str(neighborhoodlikedoncaster.shape[0]) +" suburb/neighborhood found similar to Doncaster")
neighborhoodalike.insert(0, 'Dist from Target', 0)
#neighborhoodalike.dtypes

distdiff=[]
for long, lat in zip(neighborhoodalike['Long'], neighborhoodalike['Lat']):
    dist =caldist(targetlong, targetlat, long, lat)
    #print(dist)
    distdiff.append([(dist)])

distdiffset = pd.DataFrame([item for distdiff in distdiff for item in distdiff])
distdiffset.columns = ['DistFromTarget']
distdiffset.head()
neighborhoodalike.drop(['Dist from Target'], axis=1, errors='ignore', inplace=True)
neighborhoodalike.insert(0, 'Dist from Target', distdiffset)
neighborhoodalike=neighborhoodalike.sort_values(by='Dist from Target', ascending=True)
#neighborhoodalike.dtypes
searchscope = neighborhoodalike.head(5)
searchscope=searchscope.drop([8])
searchscope=searchscope.append(neighborhoodalike.loc[neighborhoodalike['Neighborhood'] == targetneighborhood])
searchscope=searchscope.reset_index()
searchscope=searchscope.astype({'PostCode': 'int32'})
searchscope=searchscope.astype({'PostCode': 'str'})
searchscope

(180, 4)


/Users/kawingtsui/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,index,Dist from Target,Neighborhood,PostCode,Long,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,637.309802,ALBERT PARK,3206,144.950860,-37.846534,5,Café,Beach,Tram Station,Breakfast Spot,Asian Restaurant,Fishing Spot,Playground,Fast Food Restaurant,Japanese Restaurant,Italian Restaurant
1,28,637.309802,BRIGHTON,3186,144.996329,-37.915066,5,Café,Supermarket,Bakery,Movie Theater,Sushi Restaurant,Chinese Restaurant,Restaurant,Italian Restaurant,Bar,Shopping Mall
2,12,822.833747,BALWYN NORTH,3104,145.081714,-37.793695,5,Tram Station,Gym,Shopping Mall,Deli / Bodega,Zoo Exhibit,Event Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant
3,40,909.407981,BURWOOD EAST,3151,145.151119,-37.855230,5,Café,Supermarket,Burger Joint,Shopping Mall,Sandwich Place,Portuguese Restaurant,Bus Stop,Gym,Grocery Store,Department Store
4,78,NaN,DONCASTER,3108,145.125797,-37.786429,5,Café,Department Store,Dessert Shop,Multiplex,Coffee Shop,Sandwich Place,Electronics Store,Clothing Store,Supermarket,American Restaurant


<h4>Getting Suburbs and their house sell

In [479]:
#loop the neighborhood list and retrieve the price from domain.com.au

#doncasterinfo = getSuburbPerformance(client_id, client_secret, state='VIC', suburb='BURWOOD EAST', postcode='3151', category='House', rooms=3)
#doncasterinfo.head()

suburbpriceinfo = getSuburbPerformanceA(client_id, client_secret, state='VIC', suburb=searchscope['Neighborhood'], postcode=searchscope['PostCode'], long=searchscope['Long'], lat=searchscope['Lat'], category='House', rooms=3)
suburbpriceinfo=suburbpriceinfo.sort_values(by='MedianSoldPrice', ascending=True)
suburbpriceinfo
#print(suburbpriceinfo.head())

https://api.domain.com.au/v2/suburbPerformanceStatistics/VIC/ALBERT PARK/3206?propertyCategory=House&bedrooms=3&periodSize=Years&startingPeriodRelativeToCurrent=1&totalPeriods=1
[{'year': 2020, 'month': 11, 'values': {'medianSoldPrice': 2133000, 'numberSold': 38, 'highestSoldPrice': 7250000, 'lowestSoldPrice': 1000000, '5thPercentileSoldPrice': 1110000, '25thPercentileSoldPrice': 1710000, '75thPercentileSoldPrice': 2400000, '95thPercentileSoldPrice': 3020000, 'medianSaleListingPrice': 2200000, 'numberSaleListing': 51, 'highestSaleListingPrice': 7350000, 'lowestSaleListingPrice': 1400000, 'auctionNumberAuctioned': 29, 'auctionNumberSold': 15, 'auctionNumberWithdrawn': None, 'daysOnMarket': 44, 'discountPercentage': None, 'medianRentListingPrice': 950, 'numberRentListing': 92, 'highestRentListingPrice': 2200, 'lowestRentListingPrice': 550}}]
https://api.domain.com.au/v2/suburbPerformanceStatistics/VIC/BRIGHTON/3186?propertyCategory=House&bedrooms=3&periodSize=Years&startingPeriodRelative

,Neighborhood,Longtitude,Latitude,MedianSoldPrice,NumberSold,HighestSoldPrice,LowestSoldPrice,5thPercentileSoldPrice,25thPercentileSoldPrice,75thPercentileSoldPrice,95thPercentileSoldPrice
3,BURWOOD EAST,145.151119,-37.855230,992000,55,1801000,791000,850000,930000,1071000,1250000
4,DONCASTER,145.125797,-37.786429,1050000,49,1745000,595000,795000,945000,1220000,1375000
2,BALWYN NORTH,145.081714,-37.793695,1645000,60,2850000,1045000,1050000,1420000,1815000,2450000
0,ALBERT PARK,144.950860,-37.846534,2133000,38,7250000,1000000,1110000,1710000,2400000,3020000
1,BRIGHTON,144.996329,-37.915066,2143000,69,9750000,1085000,1240000,1680000,2690000,4500000


<h4>Visualizing data</h4>

In [204]:
# Importing libraries:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [488]:
#1 visualize Melbourne - Inner
address = 'Melbourne'

geolocator = Nominatim(user_agent="Melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

# create map of Toronto 
mapT = folium.Map(location=[latitude, longitude], zoom_start=11)
t=melgeo

# add markers to map
for lat, lng, borough, locality in zip(t['lat'], t['long'], t['borough'], t['locality']):
    label = '{}, {}'.format(locality, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(mapT)  
    
mapT

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [206]:
#2 visualizing k cluster
# create map
address = 'Melbourne'

geolocator = Nominatim(user_agent="Melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
displayneighborhood=neighborhoodlikedoncaster
for lat, lon, poi, cluster in zip(displayneighborhood['Lat'], displayneighborhood['Long'], displayneighborhood['Neighborhood'], displayneighborhood['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [486]:
#3 neighborhood similar to Doncaster
address = 'Melbourne'

geolocator = Nominatim(user_agent="Melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

# create map of Mel 
maptalike = folium.Map(location=[latitude, longitude], zoom_start=11)
ta=suburbpriceinfo

# scale mark according to price
pmax=ta['MedianSoldPrice'].max()
pmin=ta['MedianSoldPrice'].min()

# add markers to map
for lat, lng, locality, price in zip(ta['Latitude'], ta['Longtitude'], ta['Neighborhood'], ta['MedianSoldPrice']):
    label = '{}'.format(locality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5 + 10*(price-pmin)/(pmax-pmin),
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(maptalike)  
    
maptalike

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [487]:
ta

,Neighborhood,Longtitude,Latitude,MedianSoldPrice,NumberSold,HighestSoldPrice,LowestSoldPrice,5thPercentileSoldPrice,25thPercentileSoldPrice,75thPercentileSoldPrice,95thPercentileSoldPrice
3,BURWOOD EAST,145.151119,-37.855230,992000,55,1801000,791000,850000,930000,1071000,1250000
4,DONCASTER,145.125797,-37.786429,1050000,49,1745000,595000,795000,945000,1220000,1375000
2,BALWYN NORTH,145.081714,-37.793695,1645000,60,2850000,1045000,1050000,1420000,1815000,2450000
0,ALBERT PARK,144.950860,-37.846534,2133000,38,7250000,1000000,1110000,1710000,2400000,3020000
1,BRIGHTON,144.996329,-37.915066,2143000,69,9750000,1085000,1240000,1680000,2690000,4500000


<h4>Conclusion</h4>

We have levereage Foursquare data to locate neighborhoods that are similar to Doncaster
and locate 4 alternatives that are close to Doncestor and use domain.com.au to find the median sold price for a 3 room house as a reference
The data are visualized in map for my clients to compare.

To faciliate request to locate different neighborhood, we have defined few function to steamline the process.
